In [39]:
import pandas as pd
import numpy as np
from kmodes.kmodes import KModes
from sklearn.model_selection import cross_val_score
import sklearn
from sklearn.metrics import silhouette_score, calinski_harabaz_score 

In [6]:
df = pd.read_pickle('customer_all_variables.pkl')
df.head(5)

,Freq,Sex,Age,Age_Group,Hospital,HType,HRegion,Payer,Insurance_Type,Specific_Service,...,region_Viseu,General_Service_Appointment,General_Service_Emergency,General_Service_Examination,General_Service_Others,General_Service_Surgery,General_Service_Treatment,Sex_Feminino,Sex_Masculino,Sex_Missing
UniqueID,,,,,,,,,,,,,,,,,,,,,
00000f7264c27ba6fea0c837ed6aa0aa,4,Masculino,47.4,Prime Working,HCP,Large,Porto,MÉDIS,Private,PATOLOGIA CLINICA,...,0.0,0.2,0.2,0.60,0.0,0.00,0.0,0,1,0
00001e984eba85527fd3122056451279,8,Feminino,63.8,Mature Working,HCIS,Large,Lisbon,ALLIANZ,Private,SERVIÇOS ESPECIAIS DERMATOLOGIA,...,0.0,0.3,0.0,0.70,0.0,0.00,0.0,1,0,0
0000219e4b37d2504fb6b8c28e24a2d4,1,Feminino,10.0,Child,ICDT,Clinic,Porto,MÉDIS,Private,SERVIÇOS ESPECIAIS OFTALMOLOGIA,...,0.0,0.5,0.0,0.50,0.0,0.00,0.0,1,0,0
000026c67a83fa72aec14512887bb173,2,Masculino,37.0,Prime Working,CCC,Medium,Lisbon,MÉDIS,Private,SERVIÇOS ESPECIAIS MEDICINA DENTÁRIA,...,0.0,0.0,0.0,0.75,0.0,0.25,0.0,0,1,0
000028899fe7782862d40bb1b87807ee,2,Masculino,11.5,Child,HCD,Large,Lisbon,ADSE,State,URGÊNCIA PEDIATRIA,...,0.0,0.0,1.0,0.00,0.0,0.00,0.0,0,1,0


In [7]:
df.columns

Index(['Freq', 'Sex', 'Age', 'Age_Group', 'Hospital', 'HType', 'HRegion',
       'Payer', 'Insurance_Type', 'Specific_Service', 'Category_of_Service',
       'General_Service', 'Clinic', 'Large_Hospital', 'Medium_Hospital',
       'Payer_Missing', 'Payer_Other', 'Payer_Out_of_Pocket', 'Payer_Private',
       'Payer_Special_Program', 'Payer_State', 'region_Lisbon', 'region_Porto',
       'region_Santarem', 'region_Viseu', 'General_Service_Appointment',
       'General_Service_Emergency', 'General_Service_Examination',
       'General_Service_Others', 'General_Service_Surgery',
       'General_Service_Treatment', 'Sex_Feminino', 'Sex_Masculino',
       'Sex_Missing'],
      dtype='object')

In [19]:
# turn frequency into a categorical variable
pd.options.mode.chained_assignment = None
df['Frequency_of_visit'] = ''

df['Frequency_of_visit'][(df["Freq"]==1) ] = "Once"
df['Frequency_of_visit'][(df["Freq"]>1) &(df["Freq"]<= 5) ] = "Two to Five"
df['Frequency_of_visit'][(df["Freq"]>5) &(df["Freq"]<= 10) ] = "Six to Ten"
df['Frequency_of_visit'][(df["Freq"]>10) &(df["Freq"]<= 20) ] = "Ten to Twenty"
df['Frequency_of_visit'][(df["Freq"]>20)] = "Above Twenty"




df.head(3)

,Freq,Sex,Age,Age_Group,Hospital,HType,HRegion,Payer,Insurance_Type,Specific_Service,...,General_Service_Appointment,General_Service_Emergency,General_Service_Examination,General_Service_Others,General_Service_Surgery,General_Service_Treatment,Sex_Feminino,Sex_Masculino,Sex_Missing,Frequency_of_visit
UniqueID,,,,,,,,,,,,,,,,,,,,,
00000f7264c27ba6fea0c837ed6aa0aa,4,Masculino,47.4,Prime Working,HCP,Large,Porto,MÉDIS,Private,PATOLOGIA CLINICA,...,0.2,0.2,0.6,0.0,0.0,0.0,0,1,0,Two to Five
00001e984eba85527fd3122056451279,8,Feminino,63.8,Mature Working,HCIS,Large,Lisbon,ALLIANZ,Private,SERVIÇOS ESPECIAIS DERMATOLOGIA,...,0.3,0.0,0.7,0.0,0.0,0.0,1,0,0,Six to Ten
0000219e4b37d2504fb6b8c28e24a2d4,1,Feminino,10.0,Child,ICDT,Clinic,Porto,MÉDIS,Private,SERVIÇOS ESPECIAIS OFTALMOLOGIA,...,0.5,0.0,0.5,0.0,0.0,0.0,1,0,0,Once


In [20]:
df["Frequency_of_visit"].value_counts()

Two to Five      509188
Once             439107
Six to Ten       217658
Ten to Twenty    169259
Above Twenty     132889
Name: Frequency_of_visit, dtype: int64

In [21]:
# select categorical columns
categorical = df.select_dtypes(include=object)
categorical.head()

,Sex,Age_Group,Hospital,HType,HRegion,Payer,Insurance_Type,Specific_Service,Category_of_Service,General_Service,Frequency_of_visit
UniqueID,,,,,,,,,,,
00000f7264c27ba6fea0c837ed6aa0aa,Masculino,Prime Working,HCP,Large,Porto,MÉDIS,Private,PATOLOGIA CLINICA,PATOLOGIA CLINICA,Examination,Two to Five
00001e984eba85527fd3122056451279,Feminino,Mature Working,HCIS,Large,Lisbon,ALLIANZ,Private,SERVIÇOS ESPECIAIS DERMATOLOGIA,EXAMES ESPECIAIS,Examination,Six to Ten
0000219e4b37d2504fb6b8c28e24a2d4,Feminino,Child,ICDT,Clinic,Porto,MÉDIS,Private,SERVIÇOS ESPECIAIS OFTALMOLOGIA,CONSULTA EXTERNA,Examination,Once
000026c67a83fa72aec14512887bb173,Masculino,Prime Working,CCC,Medium,Lisbon,MÉDIS,Private,SERVIÇOS ESPECIAIS MEDICINA DENTÁRIA,EXAMES ESPECIAIS,Examination,Two to Five
000028899fe7782862d40bb1b87807ee,Masculino,Child,HCD,Large,Lisbon,ADSE,State,URGÊNCIA PEDIATRIA,URGÊNCIAS,Emergency,Two to Five


In [22]:
categorical.drop('General_Service',axis=1, inplace=True)

In [24]:
# fit kmode model with categories-only dataframe 
kmodes = KModes(n_clusters=10, n_init=5, init = 'Cao', verbose=1)
clusters = kmodes.fit_predict(categorical)

Initialization method and algorithm are deterministic. Setting n_init to 1.
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 155311, cost: 6287777.0
Run 1, iteration: 2/100, moves: 0, cost: 6287777.0


In [34]:
clusters = pd.DataFrame(clusters)
clusters[0].value_counts()

0    480700
3    153942
1    148768
2    131560
6    124373
5    100402
7     86613
8     84509
4     81537
9     75697
Name: 0, dtype: int64

In [35]:
#evaluate silhouette score based on famd dataframe
famd = pd.read_pickle('trans_famd.pkl').reset_index().drop('UniqueID', axis=1)
famd.head(3)

,0,1,2,3,4,5,6,7,8,9
0,2777.831280,-1089.606573,1083.391346,2601.628903,-1056.322349,-1565.667062,337.820770,842.202369,186.990991,-504.224420
1,3467.555696,-777.902490,-1839.019797,-15.492069,19.577773,-315.410871,-99.143818,849.925131,1050.015535,-1779.120183
2,2964.429453,1753.002911,583.171283,142.977934,-2894.996956,-196.526472,-1418.313731,498.537934,100.195773,-309.322824


In [40]:
sklearn.metrics.silhouette_score(famd,clusters[0].ravel())

0.03383712389741425

## summarize the clusters

In [41]:
allvar = pd.read_pickle('allvariables.pkl')
allvallvar.head()

,Freq,Sex,Age,Age_Group,Hospital,HType,HRegion,Payer,Insurance_Type,Specific_Service,...,General_Service_Treatment,Sex_Feminino,Sex_Masculino,Sex_Missing,Child,Early Working,Elderly,Mature Working,Prime Working,clusterID
UniqueID,,,,,,,,,,,,,,,,,,,,,
00000f7264c27ba6fea0c837ed6aa0aa,4,Masculino,47.4,Prime Working,HCP,Large,Porto,MÉDIS,Private,PATOLOGIA CLINICA,...,0.0,0,1,0,0,0,0,0,1,4
00001e984eba85527fd3122056451279,8,Feminino,63.8,Mature Working,HCIS,Large,Lisbon,ALLIANZ,Private,SERVIÇOS ESPECIAIS DERMATOLOGIA,...,0.0,1,0,0,0,0,0,1,0,7
0000219e4b37d2504fb6b8c28e24a2d4,1,Feminino,10.0,Child,ICDT,Clinic,Porto,MÉDIS,Private,SERVIÇOS ESPECIAIS OFTALMOLOGIA,...,0.0,1,0,0,1,0,0,0,0,6
000026c67a83fa72aec14512887bb173,2,Masculino,37.0,Prime Working,CCC,Medium,Lisbon,MÉDIS,Private,SERVIÇOS ESPECIAIS MEDICINA DENTÁRIA,...,0.0,0,1,0,0,0,0,0,1,3
000028899fe7782862d40bb1b87807ee,2,Masculino,11.5,Child,HCD,Large,Lisbon,ADSE,State,URGÊNCIA PEDIATRIA,...,0.0,0,1,0,1,0,0,0,0,5


In [43]:
allvar.reset_index(inplace=True)
allvar['clusterID'] = clusters[0]
allvar.head()

,UniqueID,Freq,Sex,Age,Age_Group,Hospital,HType,HRegion,Payer,Insurance_Type,...,General_Service_Treatment,Sex_Feminino,Sex_Masculino,Sex_Missing,Child,Early Working,Elderly,Mature Working,Prime Working,clusterID
0,00000f7264c27ba6fea0c837ed6aa0aa,4,Masculino,47.4,Prime Working,HCP,Large,Porto,MÉDIS,Private,...,0.0,0,1,0,0,0,0,0,1,0
1,00001e984eba85527fd3122056451279,8,Feminino,63.8,Mature Working,HCIS,Large,Lisbon,ALLIANZ,Private,...,0.0,1,0,0,0,0,0,1,0,0
2,0000219e4b37d2504fb6b8c28e24a2d4,1,Feminino,10.0,Child,ICDT,Clinic,Porto,MÉDIS,Private,...,0.0,1,0,0,1,0,0,0,0,7
3,000026c67a83fa72aec14512887bb173,2,Masculino,37.0,Prime Working,CCC,Medium,Lisbon,MÉDIS,Private,...,0.0,0,1,0,0,0,0,0,1,4
4,000028899fe7782862d40bb1b87807ee,2,Masculino,11.5,Child,HCD,Large,Lisbon,ADSE,State,...,0.0,0,1,0,1,0,0,0,0,1


In [44]:
allvar.to_pickle('kmode_allvar.pkl')

In [45]:
categorical_data = allvar.select_dtypes(include = object)
categorical_data.columns.to_list()

['UniqueID',
 'Sex',
 'Age_Group',
 'Hospital',
 'HType',
 'HRegion',
 'Payer',
 'Insurance_Type',
 'Specific_Service',
 'Category_of_Service',
 'General_Service']

In [46]:
def cluster_summary(cluster_id):
    cluster = allvar[allvar.clusterID==cluster_id]
    
    cluster_summary = cluster[categorical_data.columns].mode().to_dict(orient="records")[0]
    cluster_summary.update(cluster.mean().to_dict())
    return cluster_summary

In [ ]:
def cluster_comparison(*cluster_ids):
    summaries = []
    for cluster_id in cluster_ids:
        summaries.append(cluster_summary(cluster_id))
    return pd.DataFrame(summaries).set_index("clusterID").T

summary = cluster_comparison(0,1,2,3,4,5,6,7,8,9)

In [ ]:
summary

In [ ]:
# summary table was run on Sophie's computer
# go to heat maps to see a visualized summary table